In [1]:
from __future__ import annotations
from typing import TypeVar, Generic, List, Sequence
from copy import deepcopy
from functools import partial
from random import uniform
from statistics import mean,pstdev
from dataclasses import dataclass
import numpy as np
import rasterio as io
#from data_point import DataPoint

In [2]:
def zscores(original: Sequence[float]) -> List[float]:
  avg: float = mean(original)
  std: float = np.std(original)
  if std == 0: # return all zeros if there is no variation
    return [0] * len(original)
  return [(x - avg) / std for x in original]

In [3]:
from __future__ import annotations
from typing import Iterator, Tuple, List, Iterable
from math import sqrt
class DataPoint:
  def __init__(self, initial: Iterable[float]) -> None:
    self._originals: Tuple[float, ...] = tuple(initial)
    self.dimensions: Tuple[float, ...] = tuple(initial)
  @property
  def num_dimensions(self) -> int:
    return len(self.dimensions)
  def distance(self, other: DataPoint) -> float:
    combined: Iterator[Tuple[float, float]] = zip(self.dimensions,other.dimensions)
    differences: List[float] = [(x - y) ** 2 for x, y in combined]
    return sqrt(sum(differences))
  def __eq__(self, other: object) -> bool:
    if not isinstance(other, DataPoint):
      return NotImplemented
    return self.dimensions == other.dimensions
  def __repr__(self) -> str:
    return self._originals.__repr__()

In [4]:
Point = TypeVar('Point', bound=DataPoint)
class KMeans(Generic[Point]):
  @dataclass
  class Cluster:
    points: List[Point]
    centroid: DataPoint
  def __init__(self, k: int, points: List[Point]) -> None:
    if k < 1: # k-means can't do negative or zero clusters
      raise ValueError("k must be >= 1")
    self._points: List[Point] = points
    self._zscore_normalize()
# initialize empty clusters with random centroids
    self._clusters: List[KMeans.Cluster] = []
    for _ in range(k):
      rand_point: DataPoint = self._random_point()
      cluster: KMeans.Cluster = KMeans.Cluster([i*0 - 99 for i in range(len(self._points))], rand_point)
      self._clusters.append(cluster)
  @property
  def _centroids(self) -> List[DataPoint]:
    return [x.centroid for x in self._clusters]
  def _dimension_slice(self, dimension: int) -> List[float]:
    return [x.dimensions[dimension] for x in self._points]
  def _zscore_normalize(self) -> None:
    zscored: List[List[float]] = [[] for _ in range(len(self._points))]
    for dimension in range(self._points[0].num_dimensions):
      dimension_slice: List[float] = self._dimension_slice(dimension) #PROBLEM!!!
      for index, zscore in enumerate(zscores(dimension_slice)):
        zscored[index].append(zscore)
    for i in range(len(self._points)):
      self._points[i].dimensions = tuple(zscored[i])
  def _random_point(self) -> DataPoint:
    rand_dimensions: List[float] = []
    for dimension in range(self._points[0].num_dimensions):
      values: List[float] = self._dimension_slice(dimension)
      rand_value: float = uniform(min(values), max(values))
      rand_dimensions.append(rand_value)
    return DataPoint(rand_dimensions)
  def _assign_clusters(self) -> None:
    for point in self._points:
      closest: DataPoint = min(self._centroids, key=partial(DataPoint.distance, point))
      idx: int = self._centroids.index(closest)
      cluster: KMeans.Cluster = self._clusters[idx]
      for i in range(cluster.points):
          if cluster.points[i] != -99:
            break
          else: pass            
      cluster.points[i] = point
  def _generate_centroids(self) -> None:
    for cluster in self._clusters:
      if cluster.points[0] == -99: # keep the same centroid if no points
        continue
      means: List[float] = []
    for dimension in range(cluster.points[0].num_dimensions):
      for i 
      dimension_slice: List[float] = [(p.dimensions[dimension] if p != -99 else -99) for p in cluster.points]
      means.append(mean(dimension_slice))
      cluster.centroid = DataPoint(means)
  def run(self, max_iterations: int = 100) -> List[KMeans.Cluster]:
    for iteration in range(max_iterations):
      print(f"iteration #{iteration} centroids: {[i.centroid for i in self._clusters]}")
      for cluster in self._clusters: # clear all clusters
        cluster.points.clear()
      self._assign_clusters() # find cluster each point is closest to
      old_centroids: List[DataPoint] = deepcopy(self._centroids) # record
      self._generate_centroids() # find new centroids
      if old_centroids == self._centroids: # have centroids moved?
        print(f"Converged after {iteration} iterations")
        return self._clusters
    return self._clusters

In [5]:
point1: DataPoint = DataPoint([2.0, 1.0, 1.0])
point2: DataPoint = DataPoint([2.0, 2.0, 5.0])
point3: DataPoint = DataPoint([3.0, 1.5, 2.5])
kmeans_test: KMeans[DataPoint] = KMeans(2, [point1, point2, point3])
test_clusters: List[KMeans.Cluster] = kmeans_test.run()
for index, cluster in enumerate(test_clusters):
  print(f"Cluster {index}: {cluster.points}")

iteration #0 centroids: [(2.8608246082276247, 1.2729860021609216, 3.9137059779142236), (2.9002558560424574, 1.4777180319394017, 4.111542330685096)]
iteration #1 centroids: [(2.8608246082276247, 1.2729860021609216, 3.9137059779142236), (2.0, 2.0, 5.0)]
Converged after 1 iterations
Cluster 0: [(2.0, 1.0, 1.0), (3.0, 1.5, 2.5)]
Cluster 1: [(2.0, 2.0, 5.0)]


In [ ]:
import imageio as io
from pathlib import Path
import os 
import numpy as np

os.chdir(r'/content/drive/MyDrive/Colab Notebooks/remotesensing')
def imgcomposite(name1,name2):
  before = io.imread(name1)
  print(f"{before} loaded with {before.shape} dimensions")
  after = io.imread(name2)
  print(f"{after} loaded with {after.shape} dimensions")
  composite = np.array([before[:,:,4],after[:,:,4]])
  pointarray = map(lambda x,y: DataPoint([x,y]),composite[0,:,:],composite[1,:,:])
  return pointarray


In [7]:

#!pip install rasterio
#!pip install geopandas
#import geopandas
import os
import rasterio
from rasterio.plot import show
os.chdir('/Users/timothyclark/Documents/Python/remote_sensing/kmeansproject/')
file1 = 'no-0822.img'
file2 = 'no-0907.img'
class GeoPixel(DataPoint):
  def __init__(self,coord,composite):
    super().__init__(composite)
    self.coord = coord

def parser(imgfile,band):
  a = rasterio.open(imgfile)
  pixels = a.read(band)
  a.close()
  return pixels

def composite(imgarray):
  szx,szy = imgarray[0].shape[0], imgarray[0].shape[1]
  comp = np.zeros((szx,szy,len(imgarray)),dtype='int32')
  for index,band in enumerate(imgarray):
    comp[:,:,index] = band
  lst = []
  for x in range(szx):
    for y in range(szy):
      lst += [GeoPixel((x,y),list(comp[x,y,:]))]
  return lst

z = composite([parser(file1,i+1) for i in range(6)] + [parser(file2,i+1) for i in range(6)])
#z = composite([parser(file1,5),parser(file2,5),parser(file1,4)])
print(len(z))
print(z[int(uniform(0.0,float(len(z))))])

2519673
(100, 47, 53, 58, 77, 43, 100, 48, 58, 53, 87, 53)


In [8]:
#kmeans_test2: KMeans[GeoPixel] = KMeans(3,composite(parser(file1,5),parser(file2,5)))
kmeans_test2: KMeans[GeoPixel] = KMeans(5,z)
clusters: KMeans.Cluster = kmeans_test2.run()
for index, cluster in enumerate(clusters):
  print(f"Cluster {index}: {len(cluster.points)}")

iteration #0 centroids: [(92.49448027749253, 168.6227554988403, 14.853168562200631, 65.28172710535414, 66.59035482510161, 223.2571775947441, 221.02455282353841, 186.048338183384, 43.473071421304304, 254.39919278097048, 72.73160086281713, 132.43813848602903), (94.74782369968574, 108.05885114300001, 146.98982317506417, 165.45708329666593, 139.47714589504514, 247.07877149324375, 181.93880083932117, 144.7565817101614, 171.6347701442887, 161.59338911304735, 194.2085481658364, 122.73738369756529), (19.344798547371568, 153.73112519055303, 80.4815576307986, 116.22107563105159, 36.769085595631225, 47.009318292743316, 196.58591523369654, 218.19628960103006, 26.872404392138797, 185.8936438676037, 224.2799729930443, 72.12613337037442), (169.13231994098888, 38.70416300896054, 194.11784071669868, 224.59226549875254, 235.72076893219943, 52.062989061862005, 37.059588643789255, 104.41067975680765, 82.54827299854115, 174.79751732485286, 241.08669992485915, 206.014075762769), (6.261832086571342, 112.5430

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
colors = ['r','b','g','k','r']
for index, cluster in enumerate(clusters):
  y = [point.coord[0] for point in cluster.points]
  x = [point.coord[1] for point in cluster.points]
  plt.scatter(x,y,marker='.',color=colors.pop())

plt.show()


In [ ]:
import matplotlib.pyplot as plt
colors = ['r','b','g','k','r','b','g','k','r','b']
for index, cluster in enumerate(clusters):
  x = [point._originals[4] for point in cluster.points]
  y = [point._originals[10] for point in cluster.points]
  plt.scatter(x,y,color=colors.pop())

plt.show()

In [ ]:
import rasterio as io
#params = rasterio.open(file1,'r')
##z = parser(file1,5)
#sz = z.shape
#newraster = np.zeros((sz[0],sz[1],3),dtype='uint8')
#for index, cluster in enumerate(clusters):
#    for point in cluster.points:
#        newraster[point.coord[0],point.coord[1],index] = 255
        
def class2array(refimg,clusters,newfile):
    params = io.open(refimg,'r')
    sz = params.read(1).shape
    newraster = np.zeros((sz[0],sz[1],3),dtype=np.int8)
    for index, cluster in enumerate(clusters):
        for point in cluster.points:
            newraster[point.coord[0],point.coord[1],:]= (255/len(clusters)-1)*index
    write = io.open(f'{newfile}-kemans.tif','w',
        driver='GTiff',
        height=newraster.shape[0],
        width=newraster.shape[1],
        count=3,
        dtype=z.dtype,
        crs=params.crs,
        transform=params.transform)
    for h in range(index):
        write.write(newraster[:,:,h],h+1)
    params.close()
    write.close()
    return newraster

n = class2array(file1,clusters,'10-means.tiff')
print(n[50:55,50:55,:])
